# 1. Import the necessary libraries for you to be able to query 4square API

In [75]:
import requests
import json
import pandas as pd
from getpass import getpass

import os
from dotenv import load_dotenv

from pymongo import MongoClient

In [6]:
load_dotenv() 

True

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [1]:
# your code here
iron_lat= 41.3977421
iron_lon=2.1902493

In [7]:
passwd = os.getenv("passwd")

In [23]:
import requests

url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=41.3977421%2C2.1902493&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3r/TpfzZa1Jt2dzQr5+9KGqJwk9feYwt5/xcu0g6pewc="
}


response = requests.get(url, headers=headers)


r = response.json()

In [39]:
coffee_names =  [r['results'][i]['name'] for i in range(len(r['results'])) ]
coffee_locations =  [r['results'][i]['location']['address'] for i in range(len(r['results'])) ]
coffee_lat = [r['results'][i]["geocodes"]["main"]['latitude'] for i in range(len(r['results'])) ]
coffee_lon = [r['results'][i]["geocodes"]["main"]['longitude'] for i in range(len(r['results'])) ]

In [42]:
coffee_places_df = pd.DataFrame({
"names":coffee_names,
"locations":coffee_locations,
"lat":coffee_lat,
"long":coffee_lon
})

coffee_places_df

,names,locations,lat,long
0,Espai Joliu,"Carrer Badajoz, 95",41.398735,2.195069
1,Skye coffee co,"Calle Pamplona, 88",41.396876,2.191489
2,Nømad Roaster's Home,"Calle de Pujades, 95",41.397977,2.195589
3,Three Marks Coffee,"Ausiàs Marc, 151",41.396989,2.183409
4,Súper coffee & food store,"Carrer de Roc Boronat, 102",41.402197,2.195696
5,Santa Fe,"Lington, 17",41.392081,2.185666
6,Syra Coffee Poblenou,"Calle de Pujades, 100",41.396518,2.194170
7,Brew Coffee,"Calle de Roger de Flor, 102",41.394562,2.178425
8,Orval,"Calle de Buenaventura Muñoz, 31",41.392543,2.185749
9,El Petit Princep,"Gran Vía de Les Corts Catalanes, 677",41.396371,2.177069


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [44]:
# your code here

import requests

url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=41.3977421%2C2.1902493&radius=500&limit=5"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3r/TpfzZa1Jt2dzQr5+9KGqJwk9feYwt5/xcu0g6pewc="
}

response = requests.get(url, headers=headers)


r2 = response.json()

coffee_names =  [r2['results'][i]['name'] for i in range(len(r2['results'])) ]
coffee_locations =  [r2['results'][i]['location']['address'] for i in range(len(r2['results'])) ]
coffee_lat = [r2['results'][i]["geocodes"]["main"]['latitude'] for i in range(len(r2['results'])) ]
coffee_lon = [r2['results'][i]["geocodes"]["main"]['longitude'] for i in range(len(r2['results'])) ]

coffee_places_df2 = pd.DataFrame({
"names":coffee_names,
"locations":coffee_locations,
"lat":coffee_lat,
"long":coffee_lon
})

coffee_places_df2

,names,locations,lat,long
0,Skye coffee co,"Calle Pamplona, 88",41.396876,2.191489
1,Cafeteria Botanica,"Calle D'Àlaba, 100",41.398044,2.191915
2,Syra Coffee Poblenou,"Calle de Pujades, 100",41.396518,2.194170
3,Espai Joliu,"Carrer Badajoz, 95",41.398735,2.195069
4,Bar Sancho,"Sancho de Ávila, 7",41.397037,2.187577


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [53]:
def foursquare_places (venue, coordinates):

    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates[0]}%2C{coordinates[1]}&limit=10"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3r/TpfzZa1Jt2dzQr5+9KGqJwk9feYwt5/xcu0g6pewc="
    }

    response = requests.get(url, headers=headers)


    r2 = response.json()

    names =  [r2['results'][i]['name'] for i in range(len(r2['results'])) ]
    locations =  [r2['results'][i]['location']['address'] for i in range(len(r2['results'])) ]
    lat = [r2['results'][i]["geocodes"]["main"]['latitude'] for i in range(len(r2['results'])) ]
    lon = [r2['results'][i]["geocodes"]["main"]['longitude'] for i in range(len(r2['results'])) ]

    df = pd.DataFrame({
    "names":names,
    "locations":locations,
    "lat":lat,
    "long":lon
    })

    return df

In [55]:
coffee_json = foursquare_places("starbucks",(41.39687,2.191489))

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [69]:
coffee_json

,names,locations,lat,long
0,Starbucks,"Avinguda Diagonal, 208",41.404985,2.192439
1,Starbucks,"Plaça de la Sagrada Família, 19",41.402337,2.174068
2,Starbucks,"Calle Argentería, 61",41.383506,2.181090
3,Starbucks,"Rambla de Canaletes, 109",41.383476,2.171057
4,Starbucks,"Carrer D'Aragó, 271",41.392939,2.165386
5,Starbucks,"Rambla de Catalunya, 10",41.388022,2.167940
6,Starbucks,"Carrer de Pelai, 13-39",41.385779,2.167932
7,Starbucks,"Ronda de Sant Pere, 17",41.388819,2.171884
8,Starbucks,"Moll D'Espanya, 5",41.374808,2.182781
9,Starbucks,"Rambla Canaletas, 89",41.381925,2.172230


In [73]:
# your code here
coffee_json.to_json("data/coffe_shops.json", orient="records")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [82]:
# your code here

client = MongoClient("localhost:27017")
db = client["ironhack"]
mycol = db.create_collection("ironcoffee")


In [91]:
ironcoffee = db["ironcoffee"]

with open("data\coffe_shops.json") as file:
    file_data = json.load(file)


ironcoffee.insert_many(file_data)